# Encoders mini-project:
## Implementation of a numerical sequence generator

### I. Introduction

As seen in the explanation.ipnyb notebook, enven though decoders as implemented in this repository were introduced as part of a bigger architecture - the *transformer* architecture - they can be used as a standalone architecture for sequences generation. 

In this notebook, we will implement a simple sequence generator and make different tests and observation to illustrate what was said in the explanations. 

### II. Implementation of the model

First, let's import the code from model.py. The containt of this file if precisely what was done in the explanations notebook:

In [44]:
from model import SelfAttention, TransformerBlock, StandaloneDecoderBlock, StandaloneDecoder
import torch
import torch.nn as nn
import torch.nn.functional as F

### III. Configuration

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 100
EMBED_SIZE = 256
NUM_LAYERS = 4
NUM_HEADS = 4
FORWARD_EXPANSION = 4
LEARNING_RATE = 5e-4
DROPOUT = 0.2
MAX_LENGTH=30
PAD_TOKEN_ID=11


### IV. Creation of the dataset

For this mini-project, we will make the dataset class ourself, using the data from https://gist.github.com/elifiner/cc90fdd387449158829515782936a9a4 

This modified file text is available in the same folder as this notebook. Names with accents were removed for most as well as unvalid entries for this project

In [46]:
from torch.utils.data import Dataset
import random as rd
import numpy as np
import string

class NamesDataset(Dataset):
    def __init__(self, path_to_names="first-names.txt", max_length=20, train=True):
        super().__init__()

        self.max_length = max_length

        with open(path_to_names, 'r', encoding='utf-8') as f:
            fcontent = f.read()

        self.names = fcontent.split("\n")

        # For this specific datasrt, need to clean some names:
        self.names = [name for name in self.names if not '.' in name]
        self.names = [name for name in self.names if not '2' in name]

        rd.seed(24)
        rd.shuffle(self.names)
        
        if train:
            self.names = self.names[:len(self.names)*80//100]
        else:
            self.names = self.names[len(self.names)*80//100:]

        self.vocab = list(string.ascii_lowercase) + ['é', 'à'] + ['<PAD>', '<SOS>', '<EOS>']

        self.vocab2idx = {token: i for i, token in enumerate(self.vocab)}
        self.idx2vocab = {i: token for token, i in self.vocab2idx.items()}

        self.pad_idx = self.vocab2idx['<PAD>']
        self.sos_idx = self.vocab2idx['<SOS>']  
        self.eos_idx = self.vocab2idx['<EOS>']
        self.vocab_size = len(self.vocab)

    def decode(self, indices):
        if torch.is_tensor(indices):
            indices = indices.tolist()
        
        chars = []
        for idx in indices:
            char = self.idx2vocab[idx]
            if char == '<EOS>' or char == '<PAD>':
                break
            if char != '<SOS>':
                chars.append(char)
        return ''.join(chars)

    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, idx):
        
        name = self.names[idx].lower()

        # Tokenize
        tokenized_name = [self.vocab2idx[char] for char in name]

        tokenized_seq = [self.vocab2idx['<SOS>']]+ tokenized_name + [self.vocab2idx['<EOS>']]
        
        # Pad
        tokenized_seq.extend([self.vocab2idx['<PAD>']] * (self.max_length-len(tokenized_seq)))

        # Auto-regressor:

        
        input_ids = torch.tensor(tokenized_seq[:-1], dtype=torch.long)
        target_ids = torch.tensor(tokenized_seq[1:], dtype=torch.long)

        return input_ids, target_ids

We will also need to implement the associated dataloader

In [47]:
from torch.utils.data import DataLoader

train_dataset = NamesDataset(
    train=True
)

test_dataset = NamesDataset(
    train=False
)


train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

### V. Training

In [48]:
import torch.optim as optim

if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Silicon GPU
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

device = 'cpu'

print(f"USING DEVICE: {device}")

model = StandaloneDecoder(
    trg_vocab_size=len(train_dataset.vocab),
    embed_size=EMBED_SIZE,
    num_layers=NUM_LAYERS,
    num_heads=4,
    forward_expansion=FORWARD_EXPANSION,
    dropout=DROPOUT,
    device=device,
    max_length=MAX_LENGTH
)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss() 


USING DEVICE: cpu


In [49]:
# from tqdm import tqdm

# for epoch in range(NUM_EPOCHS):
#     print(f"\n EPOCH {epoch+1}/{NUM_EPOCHS}")

#     #########################
#     ##### Train epoch: ######
#     #########################
#     model.train()

#     for batch in tqdm(train_loader, desc="Training"):
#         total_loss = 0
    
#     for batch_idx, (input_ids, target_ids) in enumerate(tqdm(train_loader, desc="Training")):
#         input_ids = input_ids.to(device)
#         target_ids = target_ids.to(device)
        
#         optimizer.zero_grad()
        
#         # Forward pass
#         logits = model(input_ids)  # (batch_size, seq_len, vocab_size)
        
#         # Reshape pour CrossEntropyLoss
#         loss = criterion(logits.reshape(-1, logits.size(-1)), target_ids.reshape(-1))
        
#         # Backward pass
#         loss.backward()
#         optimizer.step()
        
#         total_loss += loss.item()
        
#         # Log périodique
#         if (batch_idx + 1) % 50 == 0:
#             avg_loss = total_loss / (batch_idx + 1)
#             print(f"  Batch {batch_idx+1}, Loss: {avg_loss:.4f}")
    
#     print(f"✅ Epoch {epoch+1} terminée! Loss moyenne: {total_loss/len(train_loader):.4f}")
    

In [50]:
# %%
# 🚀 ENTRAÎNEMENT avec tests intégrés à chaque epoch
from tqdm import tqdm
import torch.nn.functional as F

print("🚀 === ENTRAÎNEMENT AVEC TESTS INTÉGRÉS ===")

# Créer les DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*60}")
    print(f"🚂 EPOCH {epoch+1}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    
    #########################
    ##### TRAIN PHASE #######
    #########################
    model.train()
    train_loss = 0
    train_batches = 0
    
    for batch_idx, (input_ids, target_ids) in enumerate(tqdm(train_loader, desc="🏋️ Training")):
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        logits = model(input_ids)
        
        # Loss
        loss = criterion(logits.reshape(-1, logits.size(-1)), target_ids.reshape(-1))
        
        # Backward
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_batches += 1
    
    avg_train_loss = train_loss / train_batches
    
    #########################
    ##### TEST PHASE ########
    #########################
    model.eval()
    test_loss = 0
    test_batches = 0
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for input_ids, target_ids in tqdm(test_loader, desc="🧪 Testing"):
            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            
            # Forward pass
            logits = model(input_ids)
            
            # Loss
            loss = criterion(logits.reshape(-1, logits.size(-1)), target_ids.reshape(-1))
            test_loss += loss.item()
            test_batches += 1
            
            # Accuracy (ignorer les PAD tokens)
            predictions = torch.argmax(logits, dim=-1)
            mask = (target_ids != train_dataset.pad_idx)
            correct_predictions += ((predictions == target_ids) * mask).sum().item()
            total_predictions += mask.sum().item()
    
    avg_test_loss = test_loss / test_batches
    accuracy = correct_predictions / total_predictions * 100
    
    #########################
    ##### GÉNÉRATION TEST ###
    #########################
    print(f"\n🎯 === RÉSULTATS EPOCH {epoch+1} ===")
    print(f"📊 Train Loss: {avg_train_loss:.4f}")
    print(f"📊 Test Loss:  {avg_test_loss:.4f}")
    print(f"🎯 Accuracy:   {accuracy:.2f}%")
    
    # Test de génération de noms
    print(f"\n🎨 === GÉNÉRATION DE NOMS ===")
    model.eval()
    
    with torch.no_grad():
        # Méthode 1: Génération à partir de SOS
        print("🔤 Génération à partir de <SOS>:")
        for i in range(5):
            # Commencer avec SOS
            generated = [train_dataset.sos_idx]
            current_input = torch.tensor([generated], dtype=torch.long).to(device)
            
            # Générer caractère par caractère
            for _ in range(MAX_LENGTH-1):
                logits = model(current_input)
                
                # Prendre le dernier token prédit
                last_logits = logits[0, -1, :]
                
                # Sampling avec température pour plus de diversité
                temperature = 0.8
                probs = F.softmax(last_logits / temperature, dim=-1)
                next_token = torch.multinomial(probs, 1).item()
                
                # Arrêter si on génère EOS
                if next_token == train_dataset.eos_idx:
                    break
                    
                generated.append(next_token)
                
                # Mettre à jour l'input (garder que les derniers tokens pour éviter de dépasser MAX_LENGTH)
                current_input = torch.tensor([generated[-MAX_LENGTH:]], dtype=torch.long).to(device)
            
            # Décoder le nom généré
            generated_name = ""
            for token_idx in generated[1:]:  # Skip SOS
                if token_idx == train_dataset.eos_idx:
                    break
                if token_idx < len(train_dataset.vocab):
                    char = train_dataset.idx2vocab[token_idx]
                    if char not in ['<PAD>', '<SOS>', '<EOS>']:
                        generated_name += char
            
            print(f"   • Nom {i+1}: '{generated_name}' (longueur: {len(generated_name)})")
    
    # Méthode 2: Complétion de préfixes
    print(f"\n🔍 Complétion de préfixes:")
    prefixes = ['a', 'm', 'j', 'l', 'c']
    
    with torch.no_grad():
        for prefix in prefixes:
            # Encoder le préfixe
            prefix_tokens = [train_dataset.sos_idx]
            for char in prefix.lower():
                if char in train_dataset.vocab2idx:
                    prefix_tokens.append(train_dataset.vocab2idx[char])
            
            if len(prefix_tokens) > 1:  # Si le préfixe est valide
                current_input = torch.tensor([prefix_tokens], dtype=torch.long).to(device)
                
                # Continuer la génération
                for _ in range(MAX_LENGTH - len(prefix_tokens)):
                    logits = model(current_input)
                    last_logits = logits[0, -1, :]
                    
                    # Température plus basse pour des complétions plus cohérentes
                    temperature = 0.6
                    probs = F.softmax(last_logits / temperature, dim=-1)
                    next_token = torch.multinomial(probs, 1).item()
                    
                    if next_token == train_dataset.eos_idx:
                        break
                    
                    prefix_tokens.append(next_token)
                    current_input = torch.tensor([prefix_tokens[-MAX_LENGTH:]], dtype=torch.long).to(device)
                
                # Décoder
                completed_name = ""
                for token_idx in prefix_tokens[1:]:  # Skip SOS
                    if token_idx == train_dataset.eos_idx:
                        break
                    if token_idx < len(train_dataset.vocab):
                        char = train_dataset.idx2vocab[token_idx]
                        if char not in ['<PAD>', '<SOS>', '<EOS>']:
                            completed_name += char
                
                print(f"   • '{prefix}' → '{completed_name}'")
    
    #########################
    ##### ANALYSE PRÉDICTIONS
    #########################
    print(f"\n📈 === ANALYSE DES PRÉDICTIONS ===")
    
    # Prendre quelques échantillons du test set
    with torch.no_grad():
        test_sample = next(iter(test_loader))
        sample_inputs, sample_targets = test_sample
        sample_inputs = sample_inputs[:3].to(device)  # 3 premiers échantillons
        sample_targets = sample_targets[:3].to(device)
        
        logits = model(sample_inputs)
        predictions = torch.argmax(logits, dim=-1)
        
        for i in range(3):
            # Décoder l'input (nom original)
            input_name = ""
            for token_idx in sample_inputs[i]:
                if token_idx.item() < len(train_dataset.vocab):
                    char = train_dataset.idx2vocab[token_idx.item()]
                    if char not in ['<PAD>', '<SOS>', '<EOS>']:
                        input_name += char
            
            # Décoder la target
            target_name = ""
            for token_idx in sample_targets[i]:
                if token_idx.item() != train_dataset.pad_idx:
                    if token_idx.item() < len(train_dataset.vocab):
                        char = train_dataset.idx2vocab[token_idx.item()]
                        if char not in ['<PAD>', '<SOS>', '<EOS>']:
                            target_name += char
            
            # Décoder la prédiction
            pred_name = ""
            for token_idx in predictions[i]:
                if token_idx.item() < len(train_dataset.vocab):
                    char = train_dataset.idx2vocab[token_idx.item()]
                    if char not in ['<PAD>', '<SOS>', '<EOS>']:
                        pred_name += char
            
            print(f"   • Input: '{input_name}' | Target: '{target_name}' | Pred: '{pred_name}'")
    
    print(f"\n{'='*60}")
    
    # Early stopping simple basé sur la loss
    if epoch > 0 and avg_test_loss < 0.5:
        print(f"🎉 Convergence atteinte! Test loss < 0.5")
        break

print(f"\n🎉 === ENTRAÎNEMENT TERMINÉ ===")


🚀 === ENTRAÎNEMENT AVEC TESTS INTÉGRÉS ===

🚂 EPOCH 1/10


🧪 Testing: 100%|██████████| 51/51 [00:00<00:00, 155.32it/s]



🎯 === RÉSULTATS EPOCH 1 ===
📊 Train Loss: 1.0115
📊 Test Loss:  0.8911
🎯 Accuracy:   26.05%

🎨 === GÉNÉRATION DE NOMS ===
🔤 Génération à partir de <SOS>:
   • Nom 1: 'amaran' (longueur: 6)
   • Nom 2: 'tulnha' (longueur: 6)
   • Nom 3: 'deli' (longueur: 4)
   • Nom 4: 'jenho' (longueur: 5)
   • Nom 5: 'eropina' (longueur: 7)

🔍 Complétion de préfixes:
   • 'a' → 'arinda'
   • 'm' → 'mondin'
   • 'j' → 'jinan'
   • 'l' → 'larher'
   • 'c' → 'cranil'

📈 === ANALYSE DES PRÉDICTIONS ===
   • Input: 'cedric' | Target: 'cedric' | Pred: 'mariina'
   • Input: 'yvette' | Target: 'yvette' | Pred: 'maanian'
   • Input: 'hedwig' | Target: 'hedwig' | Pred: 'marian'


🚂 EPOCH 2/10


🧪 Testing: 100%|██████████| 51/51 [00:00<00:00, 161.59it/s]



🎯 === RÉSULTATS EPOCH 2 ===
📊 Train Loss: 0.9068
📊 Test Loss:  0.8672
🎯 Accuracy:   27.35%

🎨 === GÉNÉRATION DE NOMS ===
🔤 Génération à partir de <SOS>:
   • Nom 1: 'cavallice' (longueur: 9)
   • Nom 2: 'shozia' (longueur: 6)
   • Nom 3: 'frasenn' (longueur: 7)
   • Nom 4: 'jabllola' (longueur: 8)
   • Nom 5: 'rablia' (longueur: 6)

🔍 Complétion de préfixes:
   • 'a' → 'ashane'
   • 'm' → 'marty'
   • 'j' → 'janien'
   • 'l' → 'laris'
   • 'c' → 'challine'

📈 === ANALYSE DES PRÉDICTIONS ===
   • Input: 'cedric' | Target: 'cedric' | Pred: 'mhriinh'
   • Input: 'yvette' | Target: 'yvette' | Pred: 'maarhir'
   • Input: 'hedwig' | Target: 'hedwig' | Pred: 'marian'


🚂 EPOCH 3/10


...


🎯 === RÉSULTATS EPOCH 8 ===
📊 Train Loss: 0.8409
📊 Test Loss:  0.8402
🎯 Accuracy:   28.36%

🎨 === GÉNÉRATION DE NOMS ===
🔤 Génération à partir de <SOS>:
   • Nom 1: 'marlee' (longueur: 6)
   • Nom 2: 'bauria' (longueur: 6)
   • Nom 3: 'rabie' (longueur: 5)
   • Nom 4: 'marhen' (longueur: 6)
   • Nom 5: 'jufi' (longueur: 4)

🔍 Complétion de préfixes:
   • 'a' → 'alyster'
   • 'm' → 'marika'
   • 'j' → 'jeland'
   • 'l' → 'liki'
   • 'c' → 'chiliga'

📈 === ANALYSE DES PRÉDICTIONS ===
   • Input: 'cedric' | Target: 'cedric' | Pred: 'shliisi'
   • Input: 'yvette' | Target: 'yvette' | Pred: 'saalt'
   • Input: 'hedwig' | Target: 'hedwig' | Pred: 'sarii'


🚂 EPOCH 9/10


🧪 Testing: 100%|██████████| 51/51 [00:00<00:00, 166.49it/s]



🎯 === RÉSULTATS EPOCH 9 ===
📊 Train Loss: 0.8363
📊 Test Loss:  0.8261
🎯 Accuracy:   29.42%

🎨 === GÉNÉRATION DE NOMS ===
🔤 Génération à partir de <SOS>:
   • Nom 1: 'perly' (longueur: 5)
   • Nom 2: 'dabie' (longueur: 5)
   • Nom 3: 'gosi' (longueur: 4)
   • Nom 4: 'mari' (longueur: 4)
   • Nom 5: 'allaw' (longueur: 5)

🔍 Complétion de préfixes:
   • 'a' → 'aman'
   • 'm' → 'memiki'
   • 'j' → 'janal'
   • 'l' → 'lima'
   • 'c' → 'chrick'

📈 === ANALYSE DES PRÉDICTIONS ===
   • Input: 'cedric' | Target: 'cedric' | Pred: 'mhliink'
   • Input: 'yvette' | Target: 'yvette' | Pred: 'mainti'
   • Input: 'hedwig' | Target: 'hedwig' | Pred: 'mariin'


🚂 EPOCH 10/10


🧪 Testing: 100%|██████████| 51/51 [00:00<00:00, 161.19it/s]


🎯 === RÉSULTATS EPOCH 10 ===
📊 Train Loss: 0.8309
📊 Test Loss:  0.8237
🎯 Accuracy:   29.52%

🎨 === GÉNÉRATION DE NOMS ===
🔤 Génération à partir de <SOS>:
   • Nom 1: 'chelia' (longueur: 6)
   • Nom 2: 'jaivie' (longueur: 6)
   • Nom 3: 'pirmon' (longueur: 6)
   • Nom 4: 'ruzi' (longueur: 4)
   • Nom 5: 'manno' (longueur: 5)

🔍 Complétion de préfixes:
   • 'a' → 'alisha'
   • 'm' → 'margin'
   • 'j' → 'jakhen'
   • 'l' → 'lilie'
   • 'c' → 'comilo'

📈 === ANALYSE DES PRÉDICTIONS ===
   • Input: 'cedric' | Target: 'cedric' | Pred: 'saliine'
   • Input: 'yvette' | Target: 'yvette' | Pred: 'saarter'
   • Input: 'hedwig' | Target: 'hedwig' | Pred: 'salian'


🎉 === ENTRAÎNEMENT TERMINÉ ===
